# EE-374

# Chapter 6

# Chain Virtues

### 6.1 Transactions - Blocks
- Some attributes that differentiate and give security to different components are summarized in Table.6.1

<img src="images/ch061-txs-blocks.png" width="50%">

### 6.2 Safety, Revisited
- So far we've built a protocol that is safe against *collision attacks, double spending txs, witholding blocks & adversary forks*. But we can spot another vulnerability in the querying functionality of a Ledger - nodes/parties $P_1,P_2$ may not report the exact Ledger version if either one is still catching up to update the latest block while its traversing the network (within $\Delta$ time)
    - We'd like both Ledgers to be exactly the same, so they should report the same element/$\text{tx}$ at position $i$ at time $r_1$: $L_{r_1}^{P_1}[i]=L_{r_1}^{P_2}[i]$
    - **Definition 21** (Ledger Safety). *A protocol is safe if it holds that for any two parties $P_1,P_2$ and any two times $r_1,r_2$, it holds that either $L_{r_1}^{P_1}\preccurlyeq L_{r_2}^{P_2}\vee L_{r_2}^{P_2}\preccurlyeq L_{r_1}^{P_1}$*
        - where notation $A\preccurlyeq B$ for the finite sequences $A,B$ means that $B[:|A|]=A$ ie. one equence is contained within the other one, exactly to every element